In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install accelerate --upgrade
!pip install transformers --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers[torch]

In [ ]:
!pip install transformers

In [ ]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O dev-v2.0.json


--2023-09-11 01:10:19--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  --.-KB/s    in 0.1s    

2023-09-11 01:10:20 (308 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2023-09-11 01:10:20--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       1

In [ ]:
import json

def preprocess_squad(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)['data']

    processed_data = []

    for article in data:
        for paragraph in article['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                question = qa['question']
                if qa['is_impossible']:
                    answer = '<no_answer>'
                else:
                    answer = qa['answers'][0]['text']
                processed_data.append((question, context, answer))

    return processed_data

train_data = preprocess_squad('train-v2.0.json')
val_data = preprocess_squad('dev-v2.0.json')

# Print a few samples to check the preprocessing
for i in range(3):
    print(train_data[i])


('When did Beyonce start becoming popular?', 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'in the late 1990s')
('What areas did Beyonce compete in when she was growing up?', 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Hou

In [ ]:
import torch

from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Ensure that we add the pad token to the tokenizer for padding our data later
tokenizer.pad_token = tokenizer.eos_token

# Load the model configuration and initialize the model
config = GPT2Config.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2', config=config)

# Check if GPU is available and move the model to GPU if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
from transformers import DataCollatorForLanguageModeling
from torch.utils.data import Dataset, DataLoader

class SquadDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        for item in data:
            question, context, answer = item
            encoding = tokenizer(f"{question} {context} {answer}", truncation=True, max_length=512, padding='max_length', return_tensors='pt')
            self.input_ids.append(encoding['input_ids'].squeeze())
            self.attn_masks.append(encoding['attention_mask'].squeeze())

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


train_dataset = SquadDataset(train_data, tokenizer)
val_dataset = SquadDataset(val_data, tokenizer)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


In [ ]:
from torch.nn.utils.rnn import pad_sequence

class CustomDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer

    def __call__(self, batch):
        input_ids = [item[0] for item in batch]
        attn_masks = [item[1] for item in batch]

        input_ids = pad_sequence(input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id)
        attn_masks = pad_sequence(attn_masks, batch_first=True, padding_value=0)

        return {"input_ids": input_ids, "attention_mask": attn_masks, "labels": input_ids.clone()}

custom_collator = CustomDataCollator(tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,              # For demonstration purposes, you can increase this
    per_device_train_batch_size=4,   # Depending on the GPU memory, you can adjust this
    per_device_eval_batch_size=4,    # Depending on the GPU memory, you can adjust this
    logging_dir='./logs',
    logging_steps=100,
    save_steps=5000,
    eval_steps=1000,
    save_total_limit=2,
    evaluation_strategy="steps",
    remove_unused_columns=False,
    push_to_hub=False,
)

train_dataset = SquadDataset(train_data, tokenizer)
val_dataset = SquadDataset(val_data, tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=custom_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
1000,1.112700,1.159914
2000,1.113600,1.157367
3000,1.097000,1.155177
4000,1.079900,1.154754
5000,1.047900,1.155094
6000,1.017100,1.155656
7000,1.029400,1.156208
8000,1.000600,1.157573
9000,1.045300,1.158986
10000,0.990300,1.158266


TrainOutput(global_step=32580, training_loss=0.9852335387171961, metrics={'train_runtime': 40399.7849, 'train_samples_per_second': 3.226, 'train_steps_per_second': 0.806, 'total_flos': 3.4051316318208e+16, 'train_loss': 0.9852335387171961, 'epoch': 1.0})

In [ ]:
eval_results = trainer.evaluate()


In [ ]:
import math
perplexity = math.exp(eval_results["eval_loss"])
print(f"Perplexity: {perplexity}")


Perplexity: 3.2428428623516794


In [ ]:
# Define the save path
save_path = "/content/drive/My Drive/GPT2_SQuAD"

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)


('/content/drive/My Drive/GPT2_SQuAD/tokenizer_config.json',
 '/content/drive/My Drive/GPT2_SQuAD/special_tokens_map.json',
 '/content/drive/My Drive/GPT2_SQuAD/vocab.json',
 '/content/drive/My Drive/GPT2_SQuAD/merges.txt',
 '/content/drive/My Drive/GPT2_SQuAD/added_tokens.json')

In [ ]:
# Estimated Time to Run
# 1 Epoch - 4.5hr
# 2 Epoch - 9hrs
# 3 Epoch - 13.5hrs